In [1]:
import polars as pl

https://github.com/minimaxir/ai-generated-magic-cards/blob/master/mtgjson_encoder.py


In [2]:
df_identifiers = pl.read_parquet(
    "/Users/maxwoolf/Downloads/AllPrintingsParquetFiles/cardIdentifiers.parquet"
).lazy()

df_identifiers.head(2).collect()

cardKingdomEtchedId,cardKingdomFoilId,cardKingdomId,cardsphereFoilId,cardsphereId,deckboxId,mcmId,mcmMetaId,mtgArenaId,mtgjsonFoilVersionId,mtgjsonNonFoilVersionId,mtgjsonV4Id,mtgoFoilId,mtgoId,multiverseId,scryfallCardBackId,scryfallId,scryfallIllustrationId,scryfallOracleId,tcgplayerEtchedProductId,tcgplayerProductId,uuid
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
null,null,"""122719""","""19""","""20""","""2676""","""16165""","""156""",null,"""b7c19924-b4bf-56fc-aa73-f586e9…",null,"""ad41be73-582f-58ed-abd4-a88c1f…","""27501""","""27500""","""130550""","""0aeebaf5-8c7d-4636-9e82-8c2744…","""7a5cd03c-4227-4551-aa4b-7d119f…","""be2f7173-c8b7-4172-a388-9b2c6b…","""fc2ccab7-cab1-4463-b73d-898070…",null,"""15032""","""5f8287b1-5bb6-5f4c-ad17-316a40…"
null,"""123087""",null,null,null,null,"""16165""","""156""",null,null,"""5f8287b1-5bb6-5f4c-ad17-316a40…","""fcd5d3ab-d58e-5cea-861a-cb377f…",null,null,null,"""0aeebaf5-8c7d-4636-9e82-8c2744…","""82072a1d-c1ab-4b4f-875f-d05914…","""be2f7173-c8b7-4172-a388-9b2c6b…","""fc2ccab7-cab1-4463-b73d-898070…",null,"""15032""","""b7c19924-b4bf-56fc-aa73-f586e9…"


Get Sets data (will need to build a `Enum` for ordering)


In [3]:
df_sets = (
    pl.read_parquet("/Users/maxwoolf/Downloads/AllPrintingsParquetFiles/sets.parquet")
    .lazy()
    .select(["tcgplayerGroupId", "name", "code", "releaseDate"])
    .unique()
    .sort(["releaseDate", "tcgplayerGroupId"])
    .collect()
)
df_sets

tcgplayerGroupId,name,code,releaseDate
f64,str,str,str
7.0,"""Limited Edition Alpha""","""LEA""","""1993-08-05"""
17.0,"""Limited Edition Beta""","""LEB""","""1993-10-04"""
115.0,"""Unlimited Edition""","""2ED""","""1993-12-01"""
1526.0,"""Collectors' Edition""","""CED""","""1993-12-10"""
1527.0,"""Intl. Collectors' Edition""","""CEI""","""1993-12-10"""
…,…,…,…
null,"""Aetherdrift Promos""","""PDFT""","""2025-02-14"""
null,"""Year of the Snake 2025""","""PL25""","""2025-02-14"""
null,"""Aetherdrift Art Series""","""ADFT""","""2025-02-14"""


In [4]:
rarity_enum = pl.Enum(["common", "uncommon", "rare", "mythic", "bonus", "special"])
sets_enum = pl.Enum(df_sets["code"].to_list() + ["*"])

In [5]:
FIELDS = [
    "name",
    "manaCost",
    "type",
    "text",
    "power",
    "toughness",
    "loyalty",
    "rarity",
    "setCode",
]


df_cards = (
    pl.read_parquet("/Users/maxwoolf/Downloads/AllPrintingsParquetFiles/cards.parquet")
    .lazy()
    .filter(pl.col("type") != "Vanguard")
    .with_columns(
        rarity=pl.col("rarity").cast(rarity_enum),
        setCode=pl.col("setCode").cast(sets_enum),
    )
    .select(["uuid"] + FIELDS)
    .unique()
    .join(df_identifiers, how="left", on="uuid")
    .select(["scryfallId"] + FIELDS)
    .sort(["setCode", "name"])
    .collect()
)

df_cards

scryfallId,name,manaCost,type,text,power,toughness,loyalty,rarity,setCode
str,str,str,str,str,str,str,str,enum,enum
"""69c3b2a3-0daa-4d42-832d-fcdfda…","""Air Elemental""","""{3}{U}{U}""","""Creature — Elemental""","""Flying""","""4""","""4""",null,"""uncommon""","""LEA"""
"""70e7ddf2-5604-41e7-bb9d-ddd03d…","""Ancestral Recall""","""{U}""","""Instant""","""Target player draws three card…",null,null,null,"""rare""","""LEA"""
"""664b46f5-0424-4f4e-9f26-6bd2cf…","""Animate Artifact""","""{3}{U}""","""Enchantment — Aura""","""Enchant artifact\nAs long as e…",null,null,null,"""uncommon""","""LEA"""
"""8fd7861d-925f-4b4c-a4ab-60be6f…","""Animate Dead""","""{1}{B}""","""Enchantment — Aura""","""Enchant creature card in a gra…",null,null,null,"""uncommon""","""LEA"""
"""d5c83259-9b90-47c2-b48e-c7d785…","""Animate Wall""","""{W}""","""Enchantment — Aura""","""Enchant Wall\nEnchanted Wall c…",null,null,null,"""rare""","""LEA"""
…,…,…,…,…,…,…,…,…,…
"""e6f210bf-30c9-4904-9ceb-4ab180…","""Whirler Virtuoso""","""{1}{U}{R}""","""Creature — Vedalken Artificer""","""When this creature enters, you…","""2""","""3""",null,"""uncommon""","""DRC"""
"""f903afd3-01b5-4892-bd78-8da1d2…","""Wizened Mentor""","""{1}{W}""","""Creature — Zombie Cleric""","""Whenever an opponent activates…","""2""","""2""",null,"""rare""","""DRC"""
"""4afc130b-09fb-44c7-a2b3-bce428…","""Wizened Mentor""","""{1}{W}""","""Creature — Zombie Cleric""","""Whenever an opponent activates…","""2""","""2""",null,"""rare""","""DRC"""


https://stackoverflow.com/a/73427426


In [6]:
df_cards_agg = (
    df_cards.group_by(FIELDS[0:7])
    .agg(rarities=pl.col("rarity").unique(), sets=pl.col("setCode").unique())
    .sort("name")
)

df_cards_agg

name,manaCost,type,text,power,toughness,loyalty,rarities,sets
str,str,str,str,str,str,str,list[enum],list[enum]
"""""Ach! Hans, Run!""""","""{2}{R}{R}{G}{G}""","""Enchantment""","""At the beginning of your upkee…",null,null,null,"[""rare""]","[""UNH""]"
"""""Brims"" Barone, Midway Mobster""","""{3}{W}{B}""","""Legendary Creature — Human Rog…","""When ""Brims"" Barone, Midway Mo…","""5""","""4""",null,"[""uncommon""]","[""UNF""]"
"""""Lifetime"" Pass Holder""","""{B}""","""Creature — Zombie Guest""","""This creature enters tapped.\n…","""2""","""1""",null,"[""rare""]","[""UNF""]"
"""""Name Sticker"" Goblin""","""{2}{R}""","""Creature — Goblin Guest""","""When this creature enters from…","""2""","""2""",null,"[""common""]","[""UNF""]"
"""""Rumors of My Death . . .""""","""{2}{B}""","""Enchantment""","""{3}{B}, Exile a permanent you …",null,null,null,"[""uncommon""]","[""UST""]"
…,…,…,…,…,…,…,…,…
"""Éomer, King of Rohan""","""{3}{R}{W}""","""Legendary Creature — Human Nob…","""Double strike\nÉomer, King of …","""2""","""2""",null,"[""rare""]","[""LTC""]"
"""Éomer, Marshal of Rohan""","""{2}{R}{R}""","""Legendary Creature — Human Kni…","""Haste\nWhenever one or more ot…","""4""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]"
"""Éowyn, Fearless Knight""","""{2}{R}{W}""","""Legendary Creature — Human Kni…","""Haste\nWhen Éowyn, Fearless Kn…","""3""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]"


Join back to the original dataframe which only has the latest printings of each cards.


In [7]:
df_cards = (
    df_cards.select(["scryfallId", "name"])
    .group_by("name", maintain_order=True)
    .last()
    .join(df_cards_agg, how="left", on="name")
    .sort("name")
)

df_cards

name,scryfallId,manaCost,type,text,power,toughness,loyalty,rarities,sets
str,str,str,str,str,str,str,str,list[enum],list[enum]
"""""Ach! Hans, Run!""""","""84f2c8f5-8e11-4639-b7de-00e4a2…","""{2}{R}{R}{G}{G}""","""Enchantment""","""At the beginning of your upkee…",null,null,null,"[""rare""]","[""UNH""]"
"""""Brims"" Barone, Midway Mobster""","""68832214-2943-4253-8884-ffa490…","""{3}{W}{B}""","""Legendary Creature — Human Rog…","""When ""Brims"" Barone, Midway Mo…","""5""","""4""",null,"[""uncommon""]","[""UNF""]"
"""""Lifetime"" Pass Holder""","""42293306-aaea-4542-8df4-813823…","""{B}""","""Creature — Zombie Guest""","""This creature enters tapped.\n…","""2""","""1""",null,"[""rare""]","[""UNF""]"
"""""Name Sticker"" Goblin""","""fd1442b4-da59-4042-835f-143c8d…","""{2}{R}""","""Creature — Goblin Guest""","""When this creature enters from…","""2""","""2""",null,"[""common""]","[""UNF""]"
"""""Rumors of My Death . . .""""","""cb3587b9-e727-4f37-b4d6-1baa73…","""{2}{B}""","""Enchantment""","""{3}{B}, Exile a permanent you …",null,null,null,"[""uncommon""]","[""UST""]"
…,…,…,…,…,…,…,…,…,…
"""Éomer, King of Rohan""","""f2c11695-f22b-44d5-937c-2578f2…","""{3}{R}{W}""","""Legendary Creature — Human Nob…","""Double strike\nÉomer, King of …","""2""","""2""",null,"[""rare""]","[""LTC""]"
"""Éomer, Marshal of Rohan""","""fba68512-f536-4961-9e24-563270…","""{2}{R}{R}""","""Legendary Creature — Human Kni…","""Haste\nWhenever one or more ot…","""4""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]"
"""Éowyn, Fearless Knight""","""c1b37891-5ed9-47e4-8d2f-c2bfd8…","""{2}{R}{W}""","""Legendary Creature — Human Kni…","""Haste\nWhen Éowyn, Fearless Kn…","""3""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]"


Postprocess:

- Remove card name in text and replace with `~`.
- Set Basic Lands `sets` to `[*]`.

https://stackoverflow.com/a/78685935


In [8]:
BASIC_LANDS = ["Island", "Swamp", "Mountain", "Plains", "Forest"]

df_cards = df_cards.with_columns(
    text=pl.when(pl.col("text").str.contains(pl.col("name"), literal=True))
    .then(
        pl.col("text")
        .str.replace_all(pl.col("name").first(), "~", literal=True)
        .over("name")
    )
    .otherwise(pl.col("text")),
    sets=pl.when(pl.col("name").is_in(BASIC_LANDS))
    .then(["*"])
    .otherwise(pl.col("sets")),
)

df_cards

name,scryfallId,manaCost,type,text,power,toughness,loyalty,rarities,sets
str,str,str,str,str,str,str,str,list[enum],list[enum]
"""""Ach! Hans, Run!""""","""84f2c8f5-8e11-4639-b7de-00e4a2…","""{2}{R}{R}{G}{G}""","""Enchantment""","""At the beginning of your upkee…",null,null,null,"[""rare""]","[""UNH""]"
"""""Brims"" Barone, Midway Mobster""","""68832214-2943-4253-8884-ffa490…","""{3}{W}{B}""","""Legendary Creature — Human Rog…","""When ~ enters, put a +1/+1 cou…","""5""","""4""",null,"[""uncommon""]","[""UNF""]"
"""""Lifetime"" Pass Holder""","""42293306-aaea-4542-8df4-813823…","""{B}""","""Creature — Zombie Guest""","""This creature enters tapped.\n…","""2""","""1""",null,"[""rare""]","[""UNF""]"
"""""Name Sticker"" Goblin""","""fd1442b4-da59-4042-835f-143c8d…","""{2}{R}""","""Creature — Goblin Guest""","""When this creature enters from…","""2""","""2""",null,"[""common""]","[""UNF""]"
"""""Rumors of My Death . . .""""","""cb3587b9-e727-4f37-b4d6-1baa73…","""{2}{B}""","""Enchantment""","""{3}{B}, Exile a permanent you …",null,null,null,"[""uncommon""]","[""UST""]"
…,…,…,…,…,…,…,…,…,…
"""Éomer, King of Rohan""","""f2c11695-f22b-44d5-937c-2578f2…","""{3}{R}{W}""","""Legendary Creature — Human Nob…","""Double strike\n~ enters with a…","""2""","""2""",null,"[""rare""]","[""LTC""]"
"""Éomer, Marshal of Rohan""","""fba68512-f536-4961-9e24-563270…","""{2}{R}{R}""","""Legendary Creature — Human Kni…","""Haste\nWhenever one or more ot…","""4""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]"
"""Éowyn, Fearless Knight""","""c1b37891-5ed9-47e4-8d2f-c2bfd8…","""{2}{R}{W}""","""Legendary Creature — Human Kni…","""Haste\nWhen ~ enters, exile ta…","""3""","""4""",null,"[""rare""]","[""PLTR"", ""LTR""]"


In [9]:
# rare Basic Lands exist: https://gatherer.wizards.com/Pages/Card/Details.aspx?multiverseid=485279
df_cards.filter(pl.col("name").is_in(BASIC_LANDS))

name,scryfallId,manaCost,type,text,power,toughness,loyalty,rarities,sets
str,str,str,str,str,str,str,str,list[enum],list[enum]
"""Forest""","""fd1b6c5c-4585-47c1-82eb-a32418…",null,"""Basic Land — Forest""","""({T}: Add {G}.)""",null,null,null,"[""common"", ""rare""]","[""*""]"
"""Island""","""5a8e9b9e-4947-4b6a-b21b-6fe009…",null,"""Basic Land — Island""","""({T}: Add {U}.)""",null,null,null,"[""common"", ""rare""]","[""*""]"
"""Mountain""","""5e4952a8-4e1f-4a74-8a06-c9d633…",null,"""Basic Land — Mountain""","""({T}: Add {R}.)""",null,null,null,"[""common"", ""rare""]","[""*""]"
"""Plains""","""a1cf2f81-3041-4349-9b37-f215c4…",null,"""Basic Land — Plains""","""({T}: Add {W}.)""",null,null,null,"[""common"", ""rare""]","[""*""]"
"""Swamp""","""040d064e-b023-4750-afeb-1f58e3…",null,"""Basic Land — Swamp""","""({T}: Add {B}.)""",null,null,null,"[""common"", ""rare""]","[""*""]"


In [10]:
# Card that is in the most sets
df_cards.sort(pl.col("sets").list.len(), descending=True).head(1).to_dicts()

[{'name': 'Evolving Wilds',
  'scryfallId': '860fef67-8a14-4c67-a86f-4dcd682d1321',
  'manaCost': None,
  'type': 'Land',
  'text': '{T}, Sacrifice this land: Search your library for a basic land card, put it onto the battlefield tapped, then shuffle.',
  'power': None,
  'toughness': None,
  'loyalty': None,
  'rarities': ['common', 'rare'],
  'sets': ['PRM',
   'ROE',
   'CMD',
   'DDH',
   'F12',
   'DKA',
   'M13',
   'DDK',
   'C13',
   'M15',
   'DDN',
   'C14',
   'DDO',
   'DTK',
   'PTKDF',
   'MM2',
   'ORI',
   'DDP',
   'BFZ',
   'C15',
   'CN2',
   'C16',
   'AKH',
   'CMA',
   'C17',
   'IMA',
   'PRIX',
   'RIX',
   'CM2',
   'PANA',
   'C18',
   'M20',
   'C19',
   'SLD',
   'IKO',
   'ANB',
   'AKR',
   'ZNC',
   'PLST',
   'CMR',
   'AFR',
   'MID',
   'VOW',
   'DBL',
   'CLB',
   'DMC',
   '40K',
   'BRO',
   'BRC',
   'J22',
   'SIS',
   'MOC',
   'LTC',
   'WOE',
   'WHO',
   'LCC',
   'PIP',
   'OTC',
   'M3C',
   'BLC',
   'DSC',
   'FDN',
   'J25',
   'PIO',
  

In [11]:
df_cards.write_parquet("mtg_data.parquet")